# Analyze an EMCEE run
this will be command, bear with this notebook for now

## 1. Create the gempyor object

Just edit your config file and the path to the data repo

In [2]:
import gempyor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import copy

from matplotlib.backends.backend_pdf import PdfPages
import os
import pandas as pd
import xarray as xr
from gempyor import config
from gempyor.inference import GempyorInference
import gempyor.postprocess_inference
import multiprocessing, shutil

import os
from multiprocessing import cpu_count

# disable  operations using the MKL linear algebra.
os.environ["OMP_NUM_THREADS"] = "1"
import emcee
from multiprocessing import Pool


# Fill your config and the path to the data repository here 
project_path = "../../COVID19_Disparities"
config_filepath = "config_SMH_Rdisparity_phase_one_phase1_blk1_fixprojnpis_CA-NC_emcee.yml"

run_id = config_filepath.split("/")[-1].split(".")[0].replace("config_", "")

#### START COPY PAST FROM flepiMoP/flepimop/gempyor_pkg/src/gempyor/calibrate.py
ncpu = cpu_count()
print(f"found {ncpu} CPUs, using these")

gempyor_inference = GempyorInference(
        config_filepath=config_filepath,
        run_id=run_id,
        prefix=None,
        first_sim_index=1,
        stoch_traj_flag=False,
        rng_seed=None,
        nslots=1,
        inference_filename_prefix="global/final/",  # usually for {global or chimeric}/{intermediate or final}
        inference_filepath_suffix="",  # usually for the slot_id
        out_run_id=None,  # if out_run_id is different from in_run_id, fill this
        out_prefix=None,  # if out_prefix is different from in_prefix, fill this
        path_prefix=project_path,  # in case the data folder is on another directory
        autowrite_seir=False,
)
nsubpop = len(gempyor_inference.modinf.subpop_struct.subpop_names)
subpop_names = gempyor_inference.modinf.subpop_struct.subpop_names

print(gempyor_inference.logloss)
print(gempyor_inference.inferpar)

found 12 CPUs, using these
  gempyor >> Running ***DETERMINISTIC*** simulation;
  gempyor >> ModelInfo USA_inference_all; index: 1; run_id: SMH_Rdisparity_phase_one_phase1_blk1_fixprojnpis_CA-NC_emcee,
  gempyor >> prefix: USA_inference_all/SMH_Rdisparity_phase_one_phase1_blk1_fixprojnpis_CA-NC_emcee/;
Loaded subpops in loaded relative probablity file: 51 Intersect with seir simulation:  2 kept
Running Gempyor Inference
LogLoss: 6 statistics and 92 data points,number of NA for each statistic: 
incidD_latino    46
incidD_other      0
incidD_asian      0
incidD_black      0
incidD_white      0
incidC_white     24
incidC_black     24
incidC_other     24
incidC_asian     24
incidC_latino    61
incidC           24
incidD            0
dtype: int64
InferenceParameters: with 92 parameters: 
    seir_modifiers: 84 parameters
    outcome_modifiers: 8 parameters

LogLoss: 6 statistics and 92 data points,number of NA for each statistic: 
incidD_latino    46
incidD_other      0
incidD_asian      0


## 2. Choose how many simulations you want to compute, and load the h5 file
here change the filename

In [95]:
nsamples=20

In [6]:
filename = "../../../COVID19_Disparities/COVID19_Disparities/calib_Disparities_small-20241004_backend.h5"
sampler = emcee.backends.HDFBackend(filename, read_only=True)


chain_index = -1 # -1 for last

# This selects the n_samples slots/walkers with the highest likelihood, and take their last evaluation. 
# Debatable, but it's a start.
max_indices = np.argsort(sampler.get_log_prob()[chain_index, :])[-nsamples:]
print(sampler.get_chain().shape)
chains = sampler.get_chain()[:chain_index, max_indices, :]        # the last iteration, for selected slots
samples = sampler.get_chain()[chain_index, max_indices, :]  # the last iteration, for selected slots
llik = sampler.get_log_prob()[:chain_index, max_indices]

AttributeError: You must run the sampler with 'store == True' before accessing the results

## 3. plot the chains
this saves a .pdf files. Only plot the n_samples from before.

In [ ]:
gempyor.postprocess_inference.plot_chains(
        inferpar=gempyor_inference.inferpar, chains=chains, llik=llik, sampled_slots=None, save_to=f"{run_id}_chains2.pdf"
    )

## 4. Generate `n_samples` simulations

**THIS REMOVES MODEL_OUTPUT**

In case your cluster run did not finish, this takes the values of parameter selected with n_samples and put them in gempyor to get the simulation. Can be long if n_samples is big.

In [7]:
# Generate the simulations
shutil.rmtree("model_output/", ignore_errors=True)
shutil.rmtree(os.path.join(project_path, "model_output/"), ignore_errors=True)
gempyor_inference.set_save(True)
with multiprocessing.Pool(ncpu) as pool:
    results = pool.starmap(
        gempyor_inference.get_logloss_as_single_number, [(samples[i, :],) for i in range(len(max_indices))]
    )

In [16]:
# read the hosp files
results = []
for fn in gempyor.utils.list_filenames(folder=project_path+"/model_output/", filters=["hosp.parquet"]): ## TODO: here project path is correct, but use to concatenate path. the right way
   df = gempyor.read_df(fn)
   df = df.set_index("date")
   results.append(df)
print(len(results))

20


In [105]:
# plot the simulations with and without projections
gempyor.postprocess_inference.plot_fit(modinf=gempyor_inference.modinf, loss=gempyor_inference.logloss, list_of_df=results, save_to=f"{run_id}_fit_gtdates.pdf")
gempyor.postprocess_inference.plot_fit(modinf=gempyor_inference.modinf, loss=gempyor_inference.logloss,plot_projections=True, list_of_df=results, save_to=f"{run_id}_fit_proj.pdf")
